In [ ]:
import transformers
import torch
import gc
import pandas as pd
from tqdm.notebook import tqdm_notebook
from COT import *
import numpy as np
from tqdm import tqdm_notebook


def instantiate_pipeline_llama(access_token="<huggingface_access_token>"):
    '''
    instantiates pipeline for llama instruction tuned model
    
    Parameters:
    - access_token (str; required): huggingface access token, for more details refer to https://huggingface.co/docs/hub/en/security-tokens
    
    Returns: 
    Object: a text-generation pipeline from model "meta-llama/Meta-Llama-3.1-8B-Instruct"
    '''
    transformers.set_seed(42)
    model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    
    pipe = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        token=access_token,
        device_map="auto"
    )
    return(pipe)


def classifier(text, pipe):
    '''
    Generates topic model labels from the provided topic model's keywords (topic_modeling_keywords) 
    with few-shot in-context learning using either llama or phi pre-trained model.
    
    Parameters:
    - text (str; required): The social media post to be classified.
    - pipe (object; required): The pipeline of an instantiated pre-trained model used.
    
    Returns:
    str: Labels of topic models with explanation and reasoning.
    '''
    system_message = '''You are tasked with analyzing social media posts for signs of clinical depression strictly for classification purposes. You are not required to provide advice or intervention, only to analyze and classify based on the given criteria.'''


    task_prompt = '''
    You are tasked with analyzing social media posts for signs that may indicate clinical depression. You do not need to provide help or advice. Your role is to analyze and classify based on keywords, symptoms, or emotional expressions, without offering recommendations or assistance.

    Treat both news articles related to depression, research findings discussing depression or anxiety, and personal accounts displaying depression-related symptoms as indicative of clinical depression.

    Follow these steps:

    1. Identify if the post is a news article, research finding, or personal account.
    2. Analyze the post for any keywords, symptoms, or emotional expressions that may be linked to clinical depression (e.g., hopelessness, fatigue, sadness, anxiety, or stress).
    3. Evaluate the context:
        - Posts that describe personal experiences of depression or symptoms (e.g., "I feel hopeless," "I'm struggling with sadness") should be classified as "YES."
        - News articles or research findings directly discussing clinical depression, depression-related symptoms, or mental health issues (e.g., "depression," "anxiety") should be classified as "YES."
        - Posts that mention depression in passing or discuss depression prevention without personal or symptomatic context (e.g., "Exercise helps prevent depression") should be classified as "NO."
    4. Decide if the post is related to clinical depression by answering with a "YES" or "NO":
        - Choose "YES" if the post directly mentions clinical depression, depressive symptoms, anxiety, or emotional distress related to mental health (e.g., hopelessness, fatigue, sadness, anxiety).
        - Choose "NO" if the post discusses depression in a general or abstract way (e.g., news about preventing depression) without describing any depressive symptoms.
        - Choose "NO" if the post is empty, too short to comprehend, or incomprehensible. 
    5. Provide the keywords or symptoms detected, if any.
    6. Explain your decision briefly in one or two sentences, providing specific evidence for your classification.
    7. For posts that are ambiguous or vague, lean toward a "NO" decision unless strong evidence suggests clinical depression.

    **Important**: Do not provide any advice, support, or suggestions (e.g., hotlines). Focus strictly on analyzing the post and classifying it.

    **Output Format**:  
    Please provide your response in the following structured format:  
    `["YES/NO", "<keywords/symptoms detected>", "<brief explanation of reasoning>"]`
    '''


    
    # Few-shot learning examples
    Example_1 = '''1 in 3 adolescents report depression, anxiety but only 10 percent of parents can spot mental health issues: NUS-led study'''
    output_1 = '["YES", "depression, anxiety", "This study is directly related to mental health issues, specifically depression and anxiety."]'
    
    Example_2 = '''Eating more fruits in middle age can stave off depression later in life, Singapore study finds'''
    output_2 = '["NO", "depression", "While the post mentions depression, it focuses on a preventive measure rather than discussing actual depression symptoms."]'
    
    Example_3 = '''I feel so overwhelmed taking care of my father and aunts. They rely on me for everything, and I have no time or money for myself. Sometimes I wonder if things will ever get better.'''
    output_3 = '["YES", "overwhelm, hopelessness", "The post reflects emotional overwhelm and hopelessness, which can be indicative of depression."]'

    Example_4 = '''Long story short, my dad suffered a major stroke 2 years ago which left him bedridden in a nursing home and I rage quit my job 1 year ago. My father lost all his asset gambling in his younger days so he had no asset to his name and I have been living with his two sisters (unmarried, my aunts). Normally I should not have to worried about this, but it turns out my Dad only had $2 in his bank account after gambling the CPF he withdrew at our world class casino. He didn't have much insurance because he cancelled them to save on premiums. For what he had, Aviva denied his claim because they claim he false declared his health status on application (many years ago) and will thus "eat" his premium and terminate the policy. The only thing I managed to claim successfully was the DPS and Eldershield thanks to NTUC, but it was after a lengthy one year process to obtain all the "funny" documents they needed. Apparently taking care of a mentally incapacitated person is harder than taking care of a dead person, thanks to red tapes everywhere. You don't just walk and say, I am the son and expect everything to fall into place. There is no one unified system at all.

    Nonetheless, there was a whole lot of drama in the first year of his stroke due to the hospital, social workers and incapable relatives that requires me to attend to every single thing like I own the company, and I can take time off anytime I want to be a good son like on television. Eventually I was so burn out that everything finally forced me to rage quit my job after a falling out with management. Stuck in a contract was horrible enough but they were so eager to take advantage of the fact that I needed the job and money, instead of negotiating properly. I also dropped out of private uni because I needed cash flow to pay for all of my father's bills and that Kaplan was insensitive to my predicament by not allowing partial refund of my paid fees. Instead asking me to pay the full amount again when I am ready to re-module. I didn't take that well, so I did not extend the deferment. Let's not say I did not try to get my degree, ok? My dad rather gamble than fund my studies and when I finally saved enough for further studies, I need to pay for his medical needs. I was screwed over hard, and ultimately no money means no paper degree no matter how talented I may be. I even tried to negotiate with the company to send me for training on company time, but its always no training budget for contract staffs. And then the usual, if you go training on company time, then what we hire you for? During my years in the company, never once was I send for training. When there is a new project, they just hire someone to implement it instead of giving me the opportunity. And when its implemented, I get a few power point slides and magically I'm qualified to support the new system. It really is vomit blood.

    Although I was jobless for a year, the nice thing was that I was able to get everything settled personally like the good son I should be. Or at least, I thought it should be. But it turns out that putting him in a nursing home does not entirely resolve me of my problem. He has so many medical appointments that requires me to follow up, and if I don't attend, there are funny cases where medication is missing or not collected from the hospital. Or the appointment rescheduled to another time so I can attend. I don't really know whose fault this is since the excuse I keep hearing is that "Sorry, I don't speak English so I cannot understand", so I need to be the one to take care of everything, because once again... I am the son. This keeps taking up my time and I cannot rid myself of them, so this becomes a problem when I want to find a new job. My education level is only up to Diploma and that limits my options. The roles I quality for are pretty much IT Helpdesk / Technicians which don't really have a work life balance option due to over-outsourcing. Pay wise, they are only around $2.6k and it is pretty much a dead end career, so there is no progression path. Having to pay for my father's needs and eventually his 2 sisters as they get older, with this salary really puts me at a lost.

    I have a D7 in O'lv math and that is what really screw me over big time but unless I am really that bad at math, I absolutely do not see how I can start a family (If I even have that option) and take care of everyone with a low salary. The reason I need to take care of his 2 sisters is that they took care of me when my father didn't and they took out their savings to pay off my father's gambling debts, which he didn't even brother returning to them when he withdrew his CPF. Even now, when I am not working, for some strange reasons the government's means testing suggest that I can afford to pay $1200++ a month for my father's nursing home and this is excluding all his other medicines and specialist appointments. He has over 20+ medications and among them are anti depressive pills which I don't even have the luxury of taking. As much as I can, I have tried to make ends mean but at some point in time... there is only so much one can do. Social workers I spoke to are like, well live for yourself, not for others. Find a girlfriend! But i look at my situation and I don't exactly see how I even have the luxury to live for myself. If anything, I don't know of any girl in Singapore that would even consider my position as potential boyfriend material. The truth is, I have been fighting by myself for the last 30 years without anyone to seek advise from. Friends? Let's just say, its different from television. Everyone has their own problem and their broken fingernail is more important. The best you can get out of anyone is pretty much, a listening ear on whatsapp and some "You can do it! You should come to church and donate for Jesus's blessing."
    
    I don't blame anyone though and this really issn't a rant post. I appreciate everyone that has been in my life, whether good or bad (everything is a learning experience) but ultimately, there is only so much you can beat out of a dead horse. After considering all possible options, I truly believe that ending my life and letting the government take care of my family's needs might be the best logical choice. Alternatively, I can rob the bank and get myself caught intentionally so I can get some free re-skilling opportunities in jail. Maybe when I come out, I can become some story teller with a very dramatic bestseller book to fund my retirement. To be honest, I don't know what I am expecting from this post, but this is in a way my attempt at screaming for help. Whether I deserve help or how much help I truly deserve, I don't know. Counselors always tell stories where family of 4 struggle with a household income of $2k, and they live a very happy, satisfied life so I will be just fine if I try. But how true is this story really? The bright side is that maybe the hubby can have sex with his wife to destress, but I can't fuck anyone in my family to destress. Maybe the 2 kids will grow up and help to support the family, but my situation aren't even that. I honestly feel that, I have reached the end of the crossroad and this is the end of my bloodline.'''

    output_4 = '''["YES", "hopelessness, suicidal thoughts", "The user expresses hopelessness and mentions suicidal thoughts, which are clear signs of depression."]'''
    Example_5 = '''I recently read an article on how exercise can prevent depression in the elderly. It's fascinating how staying active improves mental health overall.'''
    output_5 = '["NO", "depression", "This mentions depression prevention but is not a post about current symptoms or direct depression indicators."]'
    Example_6='''Young, jobless and feeling hopeless in Singapore - The Straits Time'''
    output_6='["YES","hopeless","The title is centered around feelings of hopeless, which is a sign of clinical depression."]'
    # The actual input prompt for the user-supplied text
    prompt = f'''Now, analyze the following post: {text}'''
    # stepwise_1="**Important**: Do not provide any advice, support, or suggestions (e.g., hotlines). Our goal is to analyze the post and classifying it. If you did so, provide any advice, support, or suggestions, please reperform the analysis."
    # Preparing the message for the model (structured chain-of-thought prompt)
    message = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": task_prompt},
        {"role": "user", "content": Example_1},
        {"role": "assistant", "content": output_1},
        {"role": "user", "content": Example_2},
        {"role": "assistant", "content": output_2},
        {"role": "user", "content": Example_3},
        {"role": "assistant", "content": output_3},
        {"role": "user", "content": Example_4},
        {"role": "assistant", "content": output_4},
        {"role": "user", "content": Example_5},
        {"role": "assistant", "content": output_5},
        {"role": "user", "content": Example_6},
        {"role": "assistant", "content": output_6},
        {"role": "user", "content": prompt}
    ]
    pipe.tokenizer.pad_token_id=128009
    # Model inference
    outputs = pipe(
        message,
        max_new_tokens=128,
        do_sample=True,  # Deterministic generation (greedy decoding)
        temperature=0.1,
        pad_token_id=pipe.tokenizer.pad_token_id,
        eos_token_id=pipe.tokenizer.eos_token_id
    )
    

    gc.collect()
    torch.cuda.empty_cache()
    
    # Return the generated label from the output
    return outputs[0]["generated_text"][-1]["content"]


In [1]:

# Read the compressed CSV file
df = pd.read_csv('data/data_SG_reddit.csv.gz', compression='gzip')
# Adjust pandas display options to show all columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)
df['text'] = df.apply(lambda row: f"Title: {row['title']}, Content: {row['selftext']}", axis=1)

C:\Users\charl\AppData\Local\Temp\ipykernel_43288\1478124110.py:6: DtypeWarning: Columns (1,4,8,12,16,17,19,21,26,30,32,34,35,38,39,40,41,42,43,44,45,46,47,51,54,55,57,58,59,60,61,62,66,67,68,69,72,73,74,77,79,81,82,83,84,85,91,92,93,94,96,97,99,100,101,102,103,104,105,106,107,109,110,111,112,113,115,116,118,120,121,123,124,126,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/data_SG_reddit.csv.gz', compression='gzip')


In [2]:
text = list(map(str, df["text"]))

In [ ]:
# define the pipe
pipe = instantiate_pipeline_llama()

# start a empty numpy array
classifications = np.empty(len(text), dtype=object)

# loop through and classify, saving every 100 iterations. 
for idx, i in tqdm_notebook(enumerate(text)):
    classifications[idx] = classifier(i, pipe)
    if (idx + 1) % 100 == 0:
        # Save intermediate results by overwriting the file
        np.save('classifications.npy', classifications)
        
# save as a numpy file
np.save('classifications.npy', classifications)

In [5]:

# saving and organizing the saved numpy classifications. 
classifications=np.load("classifications.npy", allow_pickle=True)

classify_depression = np.load('classifications.npy', allow_pickle=True)
for idx, i in tqdm_notebook(enumerate(classifications)):
    if i.startswith('[\"YES'):
        classify_depression[idx]="Yes"
    else:
        classify_depression[idx]="No"


C:\Users\charl\AppData\Local\Temp\ipykernel_43288\1561255993.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, i in tqdm_notebook(enumerate(classifications)):


0it [00:00, ?it/s]

In [6]:
## Minor cleaning
# Read the compressed CSV file
df = pd.read_csv('data/data_SG_reddit.csv.gz', compression='gzip')
# Adjust pandas display options to show all columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)
df['text'] = df.apply(lambda row: f"Title: {row['title']}, Content: {row['selftext']}", axis=1)

C:\Users\charl\AppData\Local\Temp\ipykernel_43288\2971411606.py:2: DtypeWarning: Columns (1,4,8,12,16,17,19,21,26,30,32,34,35,38,39,40,41,42,43,44,45,46,47,51,54,55,57,58,59,60,61,62,66,67,68,69,72,73,74,77,79,81,82,83,84,85,91,92,93,94,96,97,99,100,101,102,103,104,105,106,107,109,110,111,112,113,115,116,118,120,121,123,124,126,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/data_SG_reddit.csv.gz', compression='gzip')


In [8]:
df["depression_classification"]=classify_depression
df.to_csv('data/data_SG_reddit_with_classification.csv.gz', compression='gzip', index=False)